# Network Analysis

This is a set of repeated functions required everytime we're analyzing a graph. Useful for code reuse and keeping things clean.

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [32]:
def compute_network_stats(G, network_name="G"):
    if type(G) == str:
        #if its given a gml file path
        G = nx.read_gml(G)

    components = nx.connected_components(G)

    largest_cc = max(components, key=len)
    largest_cc_subgraph = G.subgraph(largest_cc)

    communities = nx.algorithms.community.louvain_communities(G)

    stats = {
        "Name": network_name,
        "|V|": G.number_of_nodes(),
        "|E|": G.number_of_edges(),
        "density": nx.density(G),
        "k": np.mean(list(dict(G.degree()).values())),
        "k weighted": np.mean(list(dict(G.degree(weight='weight')).values())),
        "|components|": nx.number_connected_components(G),
        "cc": nx.average_clustering(G),
        "s_path": nx.average_shortest_path_length(largest_cc_subgraph),
        "d": nx.diameter(largest_cc_subgraph),
        "|communities|": len(list(communities)),
        "Q": nx.community.modularity(G, communities),
    }

    stats_df = pd.DataFrame(stats, index=[0])
    stats_df.set_index("Name", inplace=True)
    
    return stats_df

def plot_degree_distribution(G, weighted=False):
    if weighted:
        degrees = dict(G.degree(weight='weight'))
    else:
        degrees = dict(G.degree())
    degree_values = sorted(set(degrees.values()))
    histogram = [list(degrees.values()).count(i) / float(nx.number_of_nodes(G)) for i in degree_values]

    plt.bar(degree_values, histogram)
    plt.xlabel("Degree")
    plt.ylabel("Fraction of nodes")
    plt.title("Degree distribution")
    plt.show()

def plot_log_log(G, weighted=False):
    if weighted:
        degrees = dict(G.degree(weight='weight'))
    else:
        degrees = [G.degree(n) for n in G.nodes()]
    mean_degree = np.mean(degrees)

    log_pk = np.log(np.unique(degrees, return_counts=True)[1])
    log_k = np.log(np.unique(degrees, return_counts=True)[0])

    x,y = log_k, log_pk

    ### regression line
    slope, intercept = np.polyfit(x, y, 1)
    # print(f"SLope: {slope*-1}, intercept: {intercept}")
    r_squared = 1 - (sum((y - (slope * x + intercept))**2) / ((len(y) - 1) * np.var(y)))

    plt.title(f"log-log degree distribution         R sq: {round(r_squared,5)}")
    plt.xlabel("log(k)")
    plt.ylabel("log(p(k))")

    plt.scatter(log_k, log_pk)
    plt.plot(x, slope * x + intercept, color="red")
    plt.show()


def get_top_n_nodes_centrality(G, centrality_dict, n=10, with_values=False, centrality_name=None):
    sorted_degrees = sorted(centrality_dict.items(), key=lambda x: x[1], reverse=True)
    df = pd.DataFrame(sorted_degrees, columns=["Node", "Centrality"])
    
    if not with_values:
        df = df.drop(columns=["Centrality"])
        if not centrality_name:
            centrality_name = "Centrality"
        df.columns = [f"{centrality_name} nodes"]

    df.index += 1
    df.index.name = "Rank"
    return df[:n]

def get_top_n_nodes(G, n=10):
    degrees = dict(G.degree())
    deg_df= get_top_n_nodes_centrality(G, degrees, n=n, centrality_name="Degree")
    w_degrees = dict(G.degree(weight='weight'))
    w_deg_df = get_top_n_nodes_centrality(G, w_degrees, n=n, centrality_name="Weighted Degree")
    betweeness = nx.betweenness_centrality(G, weight='weight')
    betw_df = get_top_n_nodes_centrality(G, betweeness, n=n, centrality_name="Betweenness weighted")
    closeness = nx.closeness_centrality(G, distance='weight')
    close_df = get_top_n_nodes_centrality(G, closeness, n=n, centrality_name="Closeness weighted")
    pagerank = nx.pagerank(G, weight='weight')
    pagerank_df = get_top_n_nodes_centrality(G, pagerank, n=n, centrality_name="PageRank weighted")
    eigenvector = nx.eigenvector_centrality(G, weight='weight')
    eigenvector_df = get_top_n_nodes_centrality(G, eigenvector, n=n, centrality_name="Eigenvector weighted")
    hubs, authorities = nx.hits(G)
    hubs_df = get_top_n_nodes_centrality(G, hubs, n=n, centrality_name="Hubs")
    authorities_df = get_top_n_nodes_centrality(G, authorities, n=n, centrality_name="Authorities")

    #concatenate all at the same index
    all_df = pd.concat([deg_df, w_deg_df, betw_df, close_df, eigenvector_df, pagerank_df, hubs_df, authorities_df], axis=1)
    return all_df

# def set_node_attributes(G, attribute_df):
    

In [34]:
G=nx.karate_club_graph()
plot_log_log(G, weighted=True)
plot_degree_distribution(G, weighted=True)

TypeError: unsupported operand type(s) for /: 'dict' and 'int'

In [30]:
get_top_n_nodes(G)

,Degree nodes,Weighted Degree nodes,Betweenness weighted nodes,Closeness weighted nodes,PageRank weighted nodes,Eigenvector weighted nodes,Hubs nodes,Authorities nodes
Rank,,,,,,,,
1,33,33,0,0,33,33,33,33
2,0,0,33,33,0,2,2,2
3,32,32,19,19,32,32,32,32
4,2,2,31,31,2,0,0,0
5,1,1,32,12,1,1,1,1
6,3,23,2,20,31,8,8,8
7,31,31,24,28,23,13,13,13
8,8,3,1,1,3,23,23,23
9,13,8,17,32,5,31,31,31


In [38]:
def set_node_attributes(G:nx.Graph, clstr_attribute_df):
    '''
    Set node attributes from a dataframe to a networkx graph

    param:
    ------
    - G: nx.Graph, networkx graph
    - clstr_attribute_df: pd.DataFrame, dataframe with cluster number and attributes

    return:
    -------
    - G: nx.Graph, networkx graph with attributes set
    '''
    #make the df a dict of index clusters
    clstr_attribute_df.set_index(clstr_attribute_df.columns[0], inplace=True)

    my_dict=clstr_attribute_df[clstr_attribute_df.columns[0]].to_dict()

    attribute=clstr_attribute_df.columns[0]
    nx.set_node_attributes(G, my_dict, name=attribute)
    return G

In [ ]:
G=nx.read_graphml('../data/graphs-3/Campylobacter_coli_ciprofloxacin_cooccurence_log_odds_filtered.graphml')

In [53]:
#pangenome annot

df=pd.read_csv("../data/cluster_descriptions/cluster_pan_gene_class.csv")
df.set_index(df.columns[0], inplace=True)
my_dict = df[df.columns[0]].to_dict()
#add " " to the dict keys
my_dict = {f'"{k}"':v for k,v in my_dict.items()}
nx.set_node_attributes(G, my_dict, name="gene_class")

In [56]:
#log odds

df=pd.read_csv('../data/log_odds_nodes/Campylobacter_coli_ciprofloxacin_log_odds.csv')
df.set_index(df.columns[0], inplace=True)
my_dict = df[df.columns[0]].to_dict()
#add " " to the dict keys
my_dict = {f'"{k}"':v for k,v in my_dict.items()}
nx.set_node_attributes(G, my_dict, name="log_odds")

In [68]:
#product name

df=pd.read_csv('../data/cluster_descriptions/cluster_product.csv')
df.set_index(df.columns[0], inplace=True)
my_dict = df[df.columns[0]].to_dict()
#add " " to the dict keys
my_dict = {f'"{k}"':v for k,v in my_dict.items()}
nx.set_node_attributes(G, my_dict, name="product_name")
my_dict

{'"Cluster 0"': 'helicase, Snf2 family',
 '"Cluster 1"': 'hypothetical protein',
 '"Cluster 2"': 'hypothetical protein',
 '"Cluster 3"': 'hypothetical protein',
 '"Cluster 4"': "DNA-directed RNA polymerase beta' subunit (EC 2.7.7.6)",
 '"Cluster 5"': 'Glutamate synthase [NADPH] large chain (EC 1.4.1.13)',
 '"Cluster 6"': 'helicase, Snf2 family',
 '"Cluster 7"': 'DNA-directed RNA polymerase beta subunit (EC 2.7.7.6)',
 '"Cluster 8"': 'Type I restriction-modification system, DNA-methyltransferase subunit M (EC 2.1.1.72) / Type I restriction-modification system, specificity subunit S',
 '"Cluster 9"': 'helicase, Snf2 family',
 '"Cluster 10"': 'Type I restriction-modification system, DNA-methyltransferase subunit M (EC 2.1.1.72) / Type I restriction-modification system, specificity subunit S',
 '"Cluster 11"': 'Beta-1,3-galactosyltransferase / Beta-1,4-galactosyltransferase',
 '"Cluster 12"': 'Type I restriction-modification system, DNA-methyltransferase subunit M (EC 2.1.1.72) / Type I re

In [70]:
#patric id

df=pd.read_csv('../data/cluster_descriptions/cluster_representatives.csv')
df.set_index(df.columns[0], inplace=True)
my_dict = df[df.columns[0]].to_dict()
my_dict={f'"{k}"':v for k,v in my_dict.items()}
nx.set_node_attributes(G, my_dict, name="patric_id")
my_dict

{'"Cluster 0"': '195.2029.peg.1780',
 '"Cluster 1"': '195.2024.peg.1611',
 '"Cluster 2"': '195.2136.peg.1150',
 '"Cluster 3"': '195.2145.peg.407',
 '"Cluster 4"': '195.2024.peg.685',
 '"Cluster 5"': '195.2024.peg.1140',
 '"Cluster 6"': '195.2240.peg.1653',
 '"Cluster 7"': '195.2282.peg.946',
 '"Cluster 8"': '195.2024.peg.149',
 '"Cluster 9"': '195.2133.peg.1513',
 '"Cluster 10"': '195.2034.peg.212',
 '"Cluster 11"': '195.2118.peg.1213',
 '"Cluster 12"': '195.2102.peg.199',
 '"Cluster 13"': '195.2222.peg.937',
 '"Cluster 14"': '195.2130.peg.4',
 '"Cluster 15"': '195.2063.peg.220',
 '"Cluster 16"': '195.2138.peg.1774',
 '"Cluster 17"': '195.2179.peg.332',
 '"Cluster 18"': '195.2037.peg.1611',
 '"Cluster 19"': '195.2136.peg.1603',
 '"Cluster 20"': '195.2029.peg.727',
 '"Cluster 21"': '195.2036.peg.815',
 '"Cluster 22"': '195.2123.peg.1684',
 '"Cluster 23"': '195.2294.peg.1787',
 '"Cluster 24"': '195.2113.peg.1254',
 '"Cluster 25"': '195.2117.peg.1680',
 '"Cluster 26"': '195.2024.peg.478',

In [71]:
G.nodes(data=True)

NodeDataView({'"Cluster 0"': {'gene_class': 'accessory', 'log_odds': 1.2703651831434948, 'product_name': 'helicase, Snf2 family', 'patric_id': '195.2029.peg.1780'}, '"Cluster 80"': {'gene_class': 'accessory', 'log_odds': 1.141298669899524, 'product_name': 'Inner membrane protein forms channel for type IV secretion of T-DNA complex, VirB3 / ATPase required for both assembly of type IV secretion complex and secretion of T-DNA complex, VirB4', 'patric_id': '195.2029.peg.1806'}, '"Cluster 187"': {'gene_class': 'accessory', 'log_odds': 1.091396776877401, 'product_name': 'DNA topoisomerase III, TraE-type (EC 5.99.1.2)', 'patric_id': '195.2029.peg.1787'}, '"Cluster 383"': {'gene_class': 'accessory', 'log_odds': 0.9435153229421136, 'product_name': 'Coupling protein VirD4, ATPase required for T-DNA transfer', 'patric_id': '195.2029.peg.1793'}, '"Cluster 394"': {'gene_class': 'accessory', 'log_odds': 0.9965453264009052, 'product_name': 'hypothetical protein', 'patric_id': '195.2244.peg.1775'}, '

In [72]:
#write its as a graphml in the same location

nx.write_graphml(G, '../data/graphs-3/Campylobacter_coli_ciprofloxacin_cooccurence_log_odds_filtered_attributes.graphml')